In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,cross_val_predict,GridSearchCV,train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.svm import SVC,SVR
from sklearn.metrics import confusion_matrix,classification_report,f1_score,r2_score,mean_squared_error,explained_variance_score
from sklearn.ensemble import VotingClassifier,VotingRegressor
from sklearn.linear_model import LinearRegression,Lasso,ridge_regression,ElasticNet,RidgeClassifier
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier
import xgboost
from xgboost import XGBClassifier,cv,plot_importance,plot_tree
from xgboost.sklearn import XGBClassifier as xgb
from xgboost.sklearn import XGBRegressor as xgr

In [298]:
df=pd.read_csv('C4T-cormmscaled.csv')
df.iloc[:,1:]=np.round(df.iloc[:,1:],2)

In [301]:
df['Class']=pd.cut(df['ThermalConductivity'],[0,10,5000],labels=[0,1])

In [302]:
df['Class'].value_counts()

0    374
1     51
Name: Class, dtype: int64

In [153]:
df.columns.values

array(['Formula', 'Temp', 'a/b', 'c/a', 'vol', 'den', 'sgn',
       'ThermalConductivity', 'composition', 'Al', 'Cu', 'Fe', 'Co', 'Ni',
       'Ge', 'Ag', 'Sb', 'Te', 'Ce', 'Yb', 'Pb', 'Bi', 'Number', 'anmin',
       'anmax', 'anfwm', 'andev', 'anrange', 'anrat', 'bmmin', 'bmfwm',
       'bmdev', 'eamin', 'eamax', 'eafwm', 'earange', 'eadev', 'pvemin',
       'pvemax', 'pvefwm', 'pverange', 'pvedev', 'fvefwm', 'fvedev',
       'dpmin', 'dpfwm', 'dpdev', 'avolrange', 'arhomin', 'arhorange',
       'arhodev', 'lcmin', 'lcmax', 'lcfwm', 'lcrange', 'hfmin',
       'hfrange', 'hfdev', 'shmin', 'shmax', 'shfwm', 'shdev', 'thmin',
       'thfwm', 'thrange', 'thrat', 'thdev', 'enmax', 'enfwm', 'enrat',
       'endev', 'iemin', 'iemax', 'iefwm', 'ierange', 'iedev', 'mtmin',
       'mtfwm', 'mtdev', 'mtrange', 'crmin', 'crfwm', 'crdev', 'crrange',
       'deb0max', 'deb0fwm', 'deb0range', 'deb0fracmax', 'deb1max',
       'deb1fwm', 'deb1range', 'deb1fracmax', 'tvemin', 'tvemax',
       'tvefwm',

In [304]:
df=df.sample(frac=1) 
X=df.drop(columns=['Formula','ThermalConductivity','composition', 'Class'])
y=df['Class']

In [244]:
rf=RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=12,min_samples_split=12,min_samples_leaf=5, max_features='log2',max_samples=0.8,class_weight='balanced',ccp_alpha=0.0015)
#help(rf)

In [280]:
y_pred1=cross_val_predict(rf,X,y,cv=6)
pd.DataFrame(classification_report(y,y_pred1,output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.948649,0.581818,0.901176,0.765233,0.904629
recall,0.938503,0.627451,0.901176,0.782977,0.901176
f1-score,0.943548,0.603774,0.901176,0.773661,0.902775
support,374.000000,51.000000,0.901176,425.000000,425.000000


In [281]:
pd.DataFrame(confusion_matrix(y,y_pred1),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,351,23
Actual high,19,32


In [43]:
n_estimators=[71,81,92,105]
min_sl=[4,5,7]
max_depth=[8,12,15]
min_sp=[8,10,12,15]
max_features=['sqrt','log2',0.75]
max_samples=[0.8,0.9]
ccp_alpha=[0.001,0.0015,0.002]
param_grid={'max_samples':max_samples,'max_depth':max_depth,'min_samples_leaf':min_sl,'min_samples_split':min_sp,'max_features':max_features,'ccp_alpha':ccp_alpha}

In [44]:
grid=GridSearchCV(estimator=rf,param_grid=param_grid,cv=5,verbose=3)

In [45]:
Grid=grid.fit(X,y)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 2/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 3/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, min_samples_split=10; total time=   0.2s
[CV 2/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=4, mi

[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.8, min_samples_leaf=7, min_samples_split=15; total time=   0.2s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 2/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 3/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=10; total time=   0.2s
[CV 2/5] END ccp_alpha=0.

[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=sqrt, max_samples=0.9, min_samples_leaf=7, min_samples_split=15; total time=   0.2s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 2/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 3/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=10; total time=   0.1s
[CV 2/5] END ccp_alpha=0.

[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=7, min_samples_split=15; total time=   0.1s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.8, min_samples_leaf=7, min_samples_split=15; total time=   0.1s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 2/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 3/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.1s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=8; total time=   0.2s
[CV 1/5] END ccp_alpha=0.

[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=7, min_samples_split=15; total time=   0.2s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=log2, max_samples=0.9, min_samples_leaf=7, min_samples_split=15; total time=   0.2s
[CV 1/5] END ccp_alpha=0.001, max_depth=8, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.4s
[CV 2/5] END ccp_alpha=0.001, max_depth=8, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.4s
[CV 3/5] END ccp_alpha=0.001, max_depth=8, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.4s
[CV 4/5] END ccp_alpha=0.001, max_depth=8, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.4s
[CV 5/5] END ccp_alpha=0.001, max_depth=8, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=8; total time=   0.4s
[CV 1/5] END ccp_alpha=0.

...

[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.2s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 2/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=sqrt, max_samples=0.9, min_samples_leaf=5, min_samples_split=10; total time=   0.2s
[CV 2/5] END ccp_a

[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=4, min_samples_split=15; total time=   0.1s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.1s
[CV 2/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.1s
[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.1s
[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.1s
[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.1s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.8, min_samples_leaf=5, min_samples_split=10; total time=   0.2s
[CV 2/5] END ccp_a

[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.2s
[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.2s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 2/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=log2, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.2s
[CV 1/5] END ccp_a

[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 2/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.8, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 5/5] END ccp_

[CV 2/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 4/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 5/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=4, min_samples_split=15; total time=   0.4s
[CV 1/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 2/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 3/5] END ccp_alpha=0.002, max_depth=15, max_features=0.75, max_samples=0.9, min_samples_leaf=5, min_samples_split=8; total time=   0.4s
[CV 4/5] END ccp

In [46]:
grid_result=Grid.cv_results_
pd.DataFrame(grid_result).sort_values(by=['rank_test_score']).head(15).to_csv('gridsearchresultrfr.csv')
pd.read_csv('gridsearchresultrfr.csv')['params'][0]

"{'ccp_alpha': 0.0015, 'max_depth': 12, 'max_features': 0.75, 'max_samples': 0.8, 'min_samples_leaf': 4, 'min_samples_split': 8}"

In [247]:
XGB=xgb(n_estimators=140,use_label_encoder=False,learning_rate=0.2,reg_lambda=0.1,reg_alpha=2,booster='gbtree',verbosity=1,max_depth=8,gamma=0.05,min_child_weight=3,scale_pos_weight=7,subsample=0.8,colsample_bytree=0.95)

In [248]:
y_predx=cross_val_predict(XGB,X,y,cv=5)
pd.DataFrame(classification_report(y,y_predx,output_dict=True))

[10:09:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:09:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:10:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:10:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

,0,1,accuracy,macro avg,weighted avg
precision,0.938830,0.571429,0.896471,0.755129,0.894742
recall,0.943850,0.549020,0.896471,0.746435,0.896471
f1-score,0.941333,0.560000,0.896471,0.750667,0.895573
support,374.000000,51.000000,0.896471,425.000000,425.000000


In [249]:
pd.DataFrame(confusion_matrix(y,y_predx),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,353,21
Actual high,23,28


In [282]:
svc=SVC(kernel='rbf',C=40,gamma=0.001,coef0=2.5,class_weight={0:1,1:7},probability=True)
#help(svc)

In [283]:
y_pred2=cross_val_predict(svc,X,y,cv=5)
pd.DataFrame(classification_report(y,y_pred2,output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.959302,0.456790,0.863529,0.708046,0.899001
recall,0.882353,0.725490,0.863529,0.803922,0.863529
f1-score,0.919220,0.560606,0.863529,0.739913,0.876186
support,374.000000,51.000000,0.863529,425.000000,425.000000


In [284]:
pd.DataFrame(confusion_matrix(y,y_pred2),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,330,44
Actual high,14,37


In [1]:
knn=KNeighborsClassifier(n_neighbors=4,weights='distance',p=1)
#help(knn)

NameError: name 'KNeighborsClassifier' is not defined

In [23]:
y_pred3=cross_val_predict(knn,X,y,cv=8)
pd.DataFrame(classification_report(y,y_pred3,output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.938144,0.729730,0.92,0.833937,0.913135
recall,0.973262,0.529412,0.92,0.751337,0.920000
f1-score,0.955381,0.613636,0.92,0.784508,0.914371
support,374.000000,51.000000,0.92,425.000000,425.000000


In [24]:
pd.DataFrame(confusion_matrix(y,y_pred3),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,364,10
Actual high,24,27


In [230]:
gb=GradientBoostingClassifier(loss='exponential',learning_rate=0.2,n_estimators=120,subsample=0.9,min_weight_fraction_leaf=0,min_samples_split=10,min_samples_leaf=5,max_depth=7,max_features='sqrt',ccp_alpha=0.00)
#help(gb)

In [285]:
y_pred3=cross_val_predict(gb,X,y,cv=8)
pd.DataFrame(classification_report(y,y_pred3,output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.942857,0.725000,0.922353,0.833929,0.916714
recall,0.970588,0.568627,0.922353,0.769608,0.922353
f1-score,0.956522,0.637363,0.922353,0.796942,0.918223
support,374.000000,51.000000,0.922353,425.000000,425.000000


In [286]:
pd.DataFrame(confusion_matrix(y,y_pred3),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,363,11
Actual high,22,29


In [293]:
estimators=[]
estimators.append(('RF',rf))
estimators.append(('SVC',svc))
#estimators.append(('KNN',knn))
estimators.append(('GB',gb))

In [294]:
vote= VotingClassifier(estimators=estimators,voting='hard',weights=[10,55,35])
#help(VotingClassifier)

In [305]:
y_pred=cross_val_predict(vote,X,y,cv=8)
pd.DataFrame(classification_report(y,y_pred,output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.964392,0.443182,0.856471,0.703787,0.901847
recall,0.868984,0.764706,0.856471,0.816845,0.856471
f1-score,0.914205,0.561151,0.856471,0.737678,0.871839
support,374.000000,51.000000,0.856471,425.000000,425.000000


In [306]:
pd.DataFrame(confusion_matrix(y,y_pred),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,325,49
Actual high,12,39


In [307]:
df['pred10']=y_pred

In [308]:
#df.to_csv('C4T-cormmscaled_pred10.csv')

In [ ]:
#final_model=vote.fit(X_train,y_train)
#y_predicted=final_model.predict(X_test)

In [306]:
#pd.DataFrame(confusion_matrix(y_predicted,y_test),index=['Actual low','Actual high'],columns=['Predicted low','Predicted high'])

,Predicted low,Predicted high
Actual low,325,49
Actual high,12,39
